In [50]:
import plotly.express as px
import kaleido
import pandas as pd
import json
import datetime
from pytimeparse.timeparse import timeparse
from datetime import timedelta

# Load rostering data
with open(f'./rostering.json', 'r', encoding='utf-8') as f:
        rostering = json.load(f)

# Load rostering data
with open(f'./scheduling_meta_input.json', 'r', encoding='utf-8') as f:
        meta = json.load(f)

shifts_duration = {}
for s in meta['shifts']:
    shifts_duration[s['id']] = datetime.timedelta(minutes= int(s['duration'][:-3]) * 60+ int(s['duration'][-2:]))

print(shifts_duration)

{'9часов день': datetime.timedelta(seconds=32400), '9часов ночь': datetime.timedelta(seconds=32400), '12часов день': datetime.timedelta(seconds=43200), '12часов ночь': datetime.timedelta(seconds=43200)}


In [51]:
# shifts to dataframe

format = '%d.%m.%y %H:%M'
shifts = map(
        lambda x: dict(Task = str(x['employeeId']),
                       Start = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format),
                       Finish = datetime.datetime.strptime(f"{x['shiftDate']} {x['shiftTimeStart']}", format) + shifts_duration[x['shiftId']],
                       Resource = x['shiftId'])
        , rostering['campainSchedule']
)

# breaks

df_shifts = pd.DataFrame(shifts)
df_shifts.head()

,Task,Start,Finish,Resource
0,6807,2022-12-02 09:30:00,2022-12-02 21:30:00,12часов день
1,6807,2022-12-03 10:45:00,2022-12-03 22:45:00,12часов день
2,6807,2022-12-05 06:15:00,2022-12-05 18:15:00,12часов день
3,6807,2022-12-06 09:30:00,2022-12-06 21:30:00,12часов день
4,6807,2022-12-08 10:00:00,2022-12-08 22:00:00,12часов день


In [52]:
# breaks to dataframe
breaks = []

for s in rostering['campainSchedule']:
    for a in s['activities']:
        employee_id = s['employeeId']
        activity_id = a['activityId']

        shift_start = datetime.datetime.strptime(f"{s['shiftDate']} {s['shiftTimeStart']}", format)
        activity_start = datetime.datetime.strptime(f"{s['shiftDate']} {a['activityTimeStart']}", format)
        activity_end = datetime.datetime.strptime(f"{s['shiftDate']} {a['activityTimeEnd']}", format)

        if activity_end < activity_start:
            # if activity start time < shift start time => its overnight activity and just +1 day
            activity_end += datetime.timedelta(days=1)

        if activity_start < shift_start:
            # if activity start time < shift start time => its overnight activity and just +1 day
            activity_start += datetime.timedelta(days=1)
            activity_end += datetime.timedelta(days=1)

        breaks.append(
            dict(Task = str(employee_id),
                 Start = activity_start,
                 Finish = activity_end,
                 Resource = activity_id)
        )

df_breaks = pd.DataFrame(breaks)
df_breaks.head()

,Task,Start,Finish,Resource
0,6807,2022-12-02 12:30:00,2022-12-02 13:00:00,12 часов день обед 1
1,6807,2022-12-02 16:45:00,2022-12-02 17:15:00,12 часов день обед 2
2,6807,2022-12-02 15:00:00,2022-12-02 15:15:00,12 часов день перерыв 1
3,6807,2022-12-02 10:30:00,2022-12-02 10:45:00,12 часов день перерыв 2
4,6807,2022-12-03 13:45:00,2022-12-03 14:15:00,12 часов день обед 1


In [53]:
df = pd.concat([df_shifts, df_breaks])
color_map = {
    '12 часов день обед 1': '#000000',
    '12 часов день обед 2': '#000000',
    '12 часов ночь обед': '#000000',
    '9 часов день обед': '#000000',
    '9 часов ночь обед': '#000000',
    '12 часов ночь перерыв 1': '#FFFFFF',
    '12 часов ночь перерыв 2': '#FFFFFF',
    '12 часов ночь перерыв 3': '#FFFFFF',
    '12 часов ночь перерыв 4': '#FFFFFF',
    '12 часов день перерыв 1': '#FFFFFF',
    '12 часов день перерыв 2': '#FFFFFF',
    '9 часов день перерыв 1': '#FFFFFF',
    '9 часов день перерыв 2': '#FFFFFF',
    '9 часов ночь перерыв 1': '#FFFFFF',
    '9 часов ночь перерыв 2': '#FFFFFF',
    '9 часов ночь перерыв 3 личн': '#FFFFFF'
}

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Resource", color_discrete_map = color_map, height = 10000)
fig.update_yaxes(autorange="reversed")
# fig.update_xaxes(rangeslider_visible=True)
fig.show()


In [54]:
# pip install --upgrade "kaleido==0.1.*"
import kaleido
import plotly.graph_objects as go
print(kaleido.__version__)

fig_export = px.timeline(df, x_start="Start", x_end="Finish", y="Task", color="Resource", color_discrete_map = color_map, height = 10000, width = 10000)
fig_export.update_yaxes(autorange="reversed")

fig_export.write_image("C:\\Projects\inline-pro\\pyworkforce\\examples\\rostering_schedule.png")

0.1.0.post1
